In [0]:
# Variables from your configuration
eh_namespace = "evhns-natraining.servicebus.windows.net"
eh_name = "evh-natraining-biju"
keyvault_scope = "dbx-ss-kv-natraining-2"
secret_name = "evh-natraining-read-write"
shared_access_key_name = "SharedAccessKeyToSendAndListen"

# Retrieve the secret (Primary Key) from Databricks Secret Scope
shared_access_key = dbutils.secrets.get(scope=keyvault_scope, key=secret_name)

# Construct the Connection String
# Note: Kafka authentication in Event Hubs uses this specific format for the password
connection_string = f"Endpoint=sb://{eh_namespace}/;SharedAccessKeyName={shared_access_key_name};SharedAccessKey={shared_access_key};EntityPath={eh_name}"

# Variables from your configuration
eh_namespace = "evhns-natraining.servicebus.windows.net"
eh_name = "evh-natraining-biju"
keyvault_scope = "dbx-ss-kv-natraining-2"
secret_name = "evh-natraining-read-write"
shared_access_key_name = "SharedAccessKeyToSendAndListen"

# Retrieve the secret (Primary Key) from Databricks Secret Scope
shared_access_key = dbutils.secrets.get(scope=keyvault_scope, key=secret_name)

# Construct the Connection String
# Note: Kafka authentication in Event Hubs uses this specific format for the password
connection_string = f"Endpoint=sb://{eh_namespace}/;SharedAccessKeyName={shared_access_key_name};SharedAccessKey={shared_access_key};EntityPath={eh_name}"


try:
    secret_value = dbutils.secrets.get(scope=keyvault_scope, key=secret_name)
    print("✓ Successfully retrieved secret from Key Vault")
except Exception as e:
    print(f"✗ Error retrieving secret: {str(e)}")
    raise

# COMMAND ----------

# MAGIC %md
# MAGIC ## Build Connection String and Kafka Options

# COMMAND ----------

# Connection string
connection_string = (
    f"Endpoint=sb://{eh_namespace}/;"
    f"SharedAccessKeyName={shared_access_key_name};"
    f"SharedAccessKey={secret_value}" )

In [0]:
KAFKA_OPTIONS = {
    "kafka.bootstrap.servers": f"{eh_namespace}:9093",
    "subscribe": eh_name,
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{connection_string}";',
    "kafka.request.timeout.ms": "60000",
    "kafka.session.timeout.ms": "30000",
    "failOnDataLoss": "false",
    "startingOffsets": "latest",
    "maxOffsetsPerTrigger": "10000"  # Process 10k records per batch
}

In [0]:
from pyspark.sql.functions import col

# Read the stream
df_stream = (spark.readStream
             .format("kafka")
             .options(**KAFKA_OPTIONS)
             .load())

# Convert binary 'value' to String for preview
df_decoded = df_stream.withColumn("body", col("value").cast("string"))

# Display the stream in the notebook
display(df_decoded.select("offset", "timestamp", "body"))

In [0]:
from pyspark.sql.functions import col

# Read the stream
df_stream = (spark.readStream
             .format("kafka")
             .options(**{'kafka.bootstrap.servers': 'localhost:9092', 'subscribe': 'topic_name'})
             .load())

# Convert binary 'value' to String for preview
df_decoded = df_stream.withColumn("body", col("value").cast("string"))

# Display the stream in the notebook
display(df_decoded.select("offset", "timestamp", "body"))

In [0]:
"evhns-natraining.servicebus.windows.net"https://evhns-natraining.servicebus.windows.net:443/

In [0]:
  %sh nc -vz evhns-natraining.servicebus.windows.net 9093

In [0]:
%sh nslookup evhns-natraining.servicebus.windows.net
%sh nc -vz evhns-natraining.servicebus.windows.net 9093

In [0]:
%sh
nc -vz evhns-natraining.servicebus.windows.net 9093
echo "nc exit code: $?"

In [0]:
from pyspark.sql.functions import col

eh_namespace = "evhns-natraining.servicebus.windows.net"
eh_name = "evh-natraining-biju"
keyvault_scope = "dbx-ss-kv-natraining-2"
secret_name = "evh-natraining-read-write"
shared_access_key_name = "SharedAccessKeyToSendAndListen"

secret_value = dbutils.secrets.get(scope=keyvault_scope, key=secret_name)

connection_string = (
    f"Endpoint=sb://{eh_namespace}/;"
    f"SharedAccessKeyName={shared_access_key_name};"
    f"SharedAccessKey={secret_value};"
    f"EntityPath={eh_name}"
)

KAFKA_OPTIONS = {
    "kafka.bootstrap.servers": f"{eh_namespace}:9093",
    "subscribe": eh_name,
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": (
        f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule '
        f'required username="$ConnectionString" password="{connection_string}";'
    ),
    "kafka.request.timeout.ms": "120000",
    "kafka.session.timeout.ms": "30000",
    "failOnDataLoss": "false",
    "startingOffsets": "latest",
    "maxOffsetsPerTrigger": "10000",
}

df_stream = (
    spark.readStream
         .format("kafka")
         .options(**KAFKA_OPTIONS)
         .load()
)

display(df_stream.select(col("offset"), col("timestamp"), col("value").cast("string")))